In [89]:
import pandas as pd
pd.set_option('max_colwidth', 100)
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix
import itertools
import scipy.stats as st
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.feature_selection import mutual_info_classif
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
import rpy2
#import pingouin as pg

import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
import rpy2.ipython.html
rpy2.ipython.html.init_printing()
from rpy2.rinterface_lib.embedded import RRuntimeError

# import R's "utils" package
utils = importr('utils')

lavaan = importr('lavaan')
semTools = importr('semTools')

In [90]:
# importing stuff the right way
try:
    semTools_version = '.'.join([str(ii) for ii in list(ro.r('packageVersion("semTools")')[0])])
except RRuntimeError:
    semTools_version = ''
# There's a bug in the main semtools libarary, this install from my fork of semtools where I've fixed it.
if semTools_version != '0.5.6.933':
    ro.r("""
        library(devtools)
        install_github("shotgunosine/semTools/semTools@aa68e38f88d9dd881617d5ade429bde2b2a74e08")
        """
    )

In [91]:
# minimal lavaan example
ro.r("""
library("lavaan")
library("semTools")

# specify the model
HS.model <- ' visual  =~ x1 + x2 + x3      
              textual =~ x4 + x5 + x6
              speed   =~ x7 + x8 + x9 '

# fit the model
fit <- cfa(HS.model, data = HolzingerSwineford1939)

# display summary output
summary(fit, fit.measures = TRUE)
""")


0,header,$lavaan.v...
1,optim,$estimato...
2,data,$ngroups ...
3,test,$standard...
4,fit,...
5,pe,...


In [92]:
# lavaan example
ro.r("""
library("lavaan")
library("semTools")
## create 3-group data in lavaan example(cfa) data
HS <- lavaan::HolzingerSwineford1939
head(HS,3)
HS$ageGroup <- ifelse(HS$ageyr < 13, "preteen",
                      ifelse(HS$ageyr > 13, "teen", "thirteen"))
## specify and fit an appropriate null model for incremental fit indices
mod.null <- c(paste0("x", 1:9, " ~ c(T", 1:9, ", T", 1:9, ", T", 1:9, ")*1"),
              paste0("x", 1:9, " ~~ c(L", 1:9, ", L", 1:9, ", L", 1:9, ")*x", 1:9))
fit.null <- cfa(mod.null, data = HS, group = "ageGroup")

## fit target model with varying levels of measurement equivalence
mod.config <- '
visual  =~ x1 + x2 + x3
textual =~ x4 + x5 + x6
speed   =~ x7 + x8 + x9
'
fit.config <- cfa(mod.config, data = HS, std.lv = TRUE, group = "ageGroup")
fit.metric <- cfa(mod.config, data = HS, std.lv = TRUE, group = "ageGroup",
                  group.equal = "loadings")
fit.scalar <- cfa(mod.config, data = HS, std.lv = TRUE, group = "ageGroup",
                  group.equal = c("loadings","intercepts"))

####################### Permutation Method
## fit indices of interest for multiparameter omnibus test
myAFIs <- c("chisq","cfi","rmsea","mfi","aic")
moreAFIs <- c("gammaHat","adjGammaHat")
## Use only 20 permutations for a demo.  In practice,
## use > 1000 to reduce sampling variability of estimated p values
## test configural invariance
set.seed(12345)
out.config <- permuteMeasEq(nPermute = 20, con = fit.config)
out.config
## test metric equivalence
set.seed(12345) # same permutations
out.metric <- permuteMeasEq(nPermute = 20, uncon = fit.config, con = fit.metric,
                            param = "loadings", AFIs = myAFIs,
                            moreAFIs = moreAFIs, null = fit.null)
summary(out.metric, nd = 4)
""")

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




  |==================================================| 100%
  |==================================================| 100%
Omnibus p value based on parametric chi-squared difference test:

Chisq diff    Df diff Pr(>Chisq) 
   14.0791    12.0000     0.2957 


Omnibus p values based on nonparametric permutation method: 

            AFI.Difference p.value
chisq              14.0791    0.50
cfi                -0.0022    0.50
rmsea              -0.0071    0.65
mfi                -0.0030    0.50
aic                -9.9209    0.50
gammaHat           -0.0013    0.50
adjGammaHat         0.0147    0.80


 Modification indices for equality constrained parameter estimates,
 with unadjusted 'p.value' based on chi-squared distribution and
 adjusted 'tukey.p.value' based on permutation distribution of the
 maximum modification index per iteration: 

    lhs op   rhs     X2 df p.value tukey.p.value flag   parameter group.lhs
1  .p1. == .p37. 1.8169  1  0.1777          0.95       visual=~x1  thirteen
2  

,,lhs,op,rhs,X2,...,group.lhs,group.rhs
0,1,.p1.,==,.p37.,1.8169024991429243,...,thirteen,preteen
1,2,.p1.,==,.p73.,1.4547462297192721,...,thirteen,teen
2,3,.p2.,==,.p38.,1.6106698869682952,...,thirteen,preteen
3,4,.p2.,==,.p74.,4.216329058513733,...,thirteen,teen
4,5,.p3.,==,.p39.,0.06966862102540441,...,thirteen,preteen
5,6,.p3.,==,.p75.,0.19363527176891665,...,thirteen,teen
6,7,.p4.,==,.p40.,0.02927082427311543,...,thirteen,preteen
7,8,.p4.,==,.p76.,1.1514857323896808,...,thirteen,teen
...,...,...,...,...,...,...,...,...
16,17,.p9.,==,.p45.,0.049187038826852346,...,thirteen,preteen


In [93]:
ro.r("""
library("lavaan")
library("semTools")
## create 3-group data in lavaan example(cfa) data
HS <- lavaan::HolzingerSwineford1939
head(HS,3)

""")

,,id,sex,ageyr,agemo,...,x8,x9
0,1,1,1,13,1,...,5.75,6.3611111
1,2,2,2,13,7,...,6.25,7.9166667
2,3,3,2,13,1,...,3.9,4.4166667


# ------------------------------------------------------------------------------------

In [94]:
# we need to import 
# -> HiTOP validation sample
# -> First assessment of genpop sample

## IMPORT HITOP VALIDATION SAMPLE


In [95]:
# this is a table that aligns scale items with their corresponding questions
htqs = pd.read_excel('../data/ValSample/Internalizing-Somatoform Items_DW.xlsx', skiprows=2,
                                names=['id', 'scale', 'item', '_0', '_1', '_2', '_3']).drop(['_0', '_1', '_2', '_3'], axis=1)
htqs.head(3)

/Users/zeleninam2/Documents/projects/hitop/2-analysis-CFA/env/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:81: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


,id,scale,item
0,1,Social Anxiety,I felt shy around other people.
1,2,Health Anxiety,I kept imagining diseases I might have
2,3,Perfectionism,I demanded perfection from myself.


In [96]:
# cmqs is a dataframe with items and responses
# htcm is the same dataframe, but only displaying HiTOP items
cmqs = pd.read_csv('../data/CFA/cogmood_questions.csv')
cmqs = cmqs.rename({'Unnamed: 0': 'id'}, axis=1)
htcm = cmqs.query("measure == 'HiTOP'")
htcm.head(1)

,id,measure,page_number,page_title,subscale,Prompt,item,response 1,response 2,response 3,response 4,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27
42,3.0,HiTOP,7,NaN,anhedonic depression,Have there been significant times during the last two weeks during which the following statement...,I had very little energy.,Not at all,A little,Moderately,A lot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [97]:
# this is a lookup table for subscales, items (sentence questions), and hitop ids
item_lookup = htcm.loc[:, ['id','subscale', 'item']].merge(htqs.loc[:, ['id','item']], how='inner', on='item', suffixes=['_cm', '_ht'])
item_lookup['htid'] = 'hitop'+ item_lookup.id_ht.astype(str)
item_lookup

,id_cm,subscale,item,id_ht,htid
0,3.0,anhedonic depression,I had very little energy.,157,hitop157
1,9.0,separation insecurity,I worried that others would abandon me.,81,hitop81
2,16.0,anxious worry,Thoughts were racing through my head.,34,hitop34
3,19.0,well-being,It was easy for me to laugh.,54,hitop54
4,23.0,appetite gain,I stuffed myself with food.,243,hitop243
5,27.0,anhedonic depression,I was unable to enjoy things like I normally do.,182,hitop182
6,28.0,separation insecurity,I wanted other people to take care of me.,69,hitop69
7,41.0,anxious worry,I had a lot of nervous energy.,89,hitop89
8,43.0,separation insecurity,I wanted someone else to make decisions for me.,50,hitop50
9,47.0,social anxiety,I avoided performing or giving a talk in front of others.,129,hitop129


In [98]:
# VALIDATION SAMPLE - main dataframe with responses
responses = pd.read_excel('../data/ValSample/inters23.xlsx')
resp_cols = responses.columns[responses.columns.isin(item_lookup.htid)]
responses.head(3)

,gender,race,hispanic,current_txt,past_txt,current_med,age,hitop39,hitop77,hitop84,hitop92,hitop93,hitop123,hitop157,hitop182,hitop230,hitop246,hitop20,hitop34,hitop89,hitop203,hitop240,hitop248,hitop265,hitop120,hitop141,hitop243,hitop275,hitop109,hitop280,hitop283,hitop67,hitop142,hitop159,hitop189,hitop5,hitop66,hitop99,hitop181,hitop231,hitop21,hitop90,hitop95,hitop160,hitop254,hitop261,hitop268,hitop15,hitop104,hitop126,hitop211,hitop215,hitop257,hitop40,hitop50,hitop69,hitop81,hitop113,hitop136,hitop151,hitop197,hitop72,hitop140,hitop143,hitop220,hitop16,hitop165,hitop225,hitop247,hitop278,hitop1,hitop17,hitop114,hitop117,hitop124,hitop129,hitop204,hitop222,hitop236,hitop258,hitop9,hitop23,hitop54,hitop106,hitop149,hitop200,hitop244,hitop245,hitop250,hitop281
0,1,5,2,1,1,1,24,4,3,3,3,4,1,3,2,2,2,3,3,2,2,2,1,2,1,2,2,2,2,2,3,3,2,2,2,2,3,2,1,1,3,2,2,1,2,3,1,3,1,3,2,1,2,3,2,2,1,3,2,2,2,3,2,2,2,1,2,1,4,1,3,4,2,3,2,1,3,1,2,1,3,3,3,2,2,2,2,2,1,2
1,1,2,2,2,1,2,32,2,1,1,2,2,1,2,2,2,1,2,3,2,2,1,1,2,1,1,2,1,1,1,1,2,2,1,2,2,1,1,1,1,2,1,1,2,3,2,2,2,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,2,1,3,1,1,1,1,1,1,1,1,1,1,1,2,3,3,3,3,2,1,3,3,3
2,2,5,2,2,2,1,68,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,2,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,4,1,1,1,1,2,1,1,1,1,1,1,3,4,4,4,4,3,4,4,4,4


In [99]:
resp_cols

Index(['hitop39', 'hitop77', 'hitop84', 'hitop92', 'hitop93', 'hitop123',
       'hitop157', 'hitop182', 'hitop230', 'hitop246', 'hitop20', 'hitop34',
       'hitop89', 'hitop203', 'hitop248', 'hitop265', 'hitop120', 'hitop141',
       'hitop243', 'hitop275', 'hitop109', 'hitop280', 'hitop283', 'hitop142',
       'hitop159', 'hitop189', 'hitop5', 'hitop66', 'hitop99', 'hitop181',
       'hitop231', 'hitop21', 'hitop90', 'hitop95', 'hitop160', 'hitop254',
       'hitop261', 'hitop268', 'hitop15', 'hitop104', 'hitop126', 'hitop211',
       'hitop215', 'hitop257', 'hitop40', 'hitop50', 'hitop69', 'hitop81',
       'hitop113', 'hitop136', 'hitop151', 'hitop197', 'hitop72', 'hitop140',
       'hitop143', 'hitop220', 'hitop16', 'hitop165', 'hitop225', 'hitop247',
       'hitop278', 'hitop1', 'hitop17', 'hitop114', 'hitop117', 'hitop124',
       'hitop129', 'hitop204', 'hitop222', 'hitop236', 'hitop258', 'hitop9',
       'hitop23', 'hitop54', 'hitop106', 'hitop149', 'hitop200', 'hitop244',
  

In [100]:
scale_scores = []
for ss, df in item_lookup.groupby('subscale'):
    print(ss)
    print(df.sort_values('id_ht').htid.values)
    tmp = responses.loc[:, df.htid].sum(1)
    tmp.name = ss.replace(" ", "_").replace("/", "_").replace("-", "_")
    scale_scores.append(tmp)
scale_scores = pd.concat(scale_scores, axis=1)

anhedonic depression
['hitop39' 'hitop77' 'hitop84' 'hitop92' 'hitop93' 'hitop123' 'hitop157'
 'hitop182' 'hitop230' 'hitop246']
anxious worry
['hitop20' 'hitop34' 'hitop89' 'hitop203' 'hitop248' 'hitop265']
appetite gain
['hitop120' 'hitop141' 'hitop243' 'hitop275']
appetite loss
['hitop109' 'hitop280' 'hitop283']
cognitive problems
['hitop142' 'hitop159' 'hitop189']
hyposomnia
['hitop5' 'hitop66' 'hitop99' 'hitop181' 'hitop231']
indecisiveness
['hitop21' 'hitop90' 'hitop95']
insomnia
['hitop160' 'hitop254' 'hitop261' 'hitop268']
panic
['hitop15' 'hitop104' 'hitop126' 'hitop211' 'hitop215' 'hitop257']
separation insecurity
['hitop40' 'hitop50' 'hitop69' 'hitop81' 'hitop113' 'hitop136' 'hitop151'
 'hitop197']
shame/guilt
['hitop72' 'hitop140' 'hitop143' 'hitop220']
situational phobia
['hitop16' 'hitop165' 'hitop225' 'hitop247' 'hitop278']
social anxiety
['hitop1' 'hitop17' 'hitop114' 'hitop117' 'hitop124' 'hitop129' 'hitop204'
 'hitop222' 'hitop236' 'hitop258']
well-being
['hitop9' 'hi

In [101]:
scale_names = list(scale_scores.columns.values)
scale_names

['anhedonic_depression',
 'anxious_worry',
 'appetite_gain',
 'appetite_loss',
 'cognitive_problems',
 'hyposomnia',
 'indecisiveness',
 'insomnia',
 'panic',
 'separation_insecurity',
 'shame_guilt',
 'situational_phobia',
 'social_anxiety',
 'well_being']

In [102]:
# this is the validation data to be used
dat_validation = responses
dat_validation.head(3)

,gender,race,hispanic,current_txt,past_txt,current_med,age,hitop39,hitop77,hitop84,hitop92,hitop93,hitop123,hitop157,hitop182,hitop230,hitop246,hitop20,hitop34,hitop89,hitop203,hitop240,hitop248,hitop265,hitop120,hitop141,hitop243,hitop275,hitop109,hitop280,hitop283,hitop67,hitop142,hitop159,hitop189,hitop5,hitop66,hitop99,hitop181,hitop231,hitop21,hitop90,hitop95,hitop160,hitop254,hitop261,hitop268,hitop15,hitop104,hitop126,hitop211,hitop215,hitop257,hitop40,hitop50,hitop69,hitop81,hitop113,hitop136,hitop151,hitop197,hitop72,hitop140,hitop143,hitop220,hitop16,hitop165,hitop225,hitop247,hitop278,hitop1,hitop17,hitop114,hitop117,hitop124,hitop129,hitop204,hitop222,hitop236,hitop258,hitop9,hitop23,hitop54,hitop106,hitop149,hitop200,hitop244,hitop245,hitop250,hitop281
0,1,5,2,1,1,1,24,4,3,3,3,4,1,3,2,2,2,3,3,2,2,2,1,2,1,2,2,2,2,2,3,3,2,2,2,2,3,2,1,1,3,2,2,1,2,3,1,3,1,3,2,1,2,3,2,2,1,3,2,2,2,3,2,2,2,1,2,1,4,1,3,4,2,3,2,1,3,1,2,1,3,3,3,2,2,2,2,2,1,2
1,1,2,2,2,1,2,32,2,1,1,2,2,1,2,2,2,1,2,3,2,2,1,1,2,1,1,2,1,1,1,1,2,2,1,2,2,1,1,1,1,2,1,1,2,3,2,2,2,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,2,1,3,1,1,1,1,1,1,1,1,1,1,1,2,3,3,3,3,2,1,3,3,3
2,2,5,2,2,2,1,68,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,2,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,4,1,1,1,1,2,1,1,1,1,1,1,3,4,4,4,4,3,4,4,4,4


## IMPORT HITOP GENPOP

In [103]:
# we need to preprocess it differently - looking at the notebook_1_preprocessing but editing the code from there as needed

In [104]:
mydat_genpop_raw = pd.read_csv('../data/NIMH0007_genpop_num_OUTPUT.csv')
mydat_genpop_raw.head()

,caseid,weight,sample_type,screener_1,screener_2,screener_3_1,screener_3_2,screener_3_3,consent,FNM_Q24,FNM_Q25_1,FNM_Q25_2,FNM_Q25_3,FNM_Q25_4,FNM_Q25_5,FNM_Q25_6,FNM_Q25_955,FNM_Q25_933,FNM_Q25_open1,FNM_Q26_d,FNM_Q27_1,FNM_Q27_2,FNM_Q27_3,FNM_Q27_4,FNM_Q27_5,FNM_Q27_6,FNM_Q27_7,FNM_Q27_8,FNM_Q27_9,FNM_Q27_10,FNM_Q27_11,FNM_Q27_12,FNM_Q27_13,FNM_Q27_14,FNM_Q27_15,FNM_Q27_16,FNM_Q27_17,FNM_Q27_18,FNM_Q27_19,FNM_Q27_955,FNM_Q27_933,FNM_Q27_open1,FNM_Q28,FNM_Q29,FNM_Q30_m_1,FNM_Q30_m_2,FNM_Q30_m_3,FNM_Q30_m_4,FNM_Q30_m_5,FNM_Q30_m_6,FNM_Q30_m_7,FNM_Q30_m_8,FNM_Q30_m_955,FNM_Q30_m_933,FNM_Q30_open1,FNM_Q31_d,FNM_Q32_1,FNM_Q32_2,FNM_Q32_3,FNM_Q32_4,FNM_Q32_5,FNM_Q32_6,FNM_Q32_7,FNM_Q32_8,FNM_Q32_9,FNM_Q32_10,FNM_Q32_955,FNM_Q32_933,FNM_Q32_open1,FNM_Q33,FNM_34,FNM_Q35_m_1,FNM_Q35_m_2,FNM_Q35_m_3,FNM_Q35_m_933,FNM_Q35_open1,FNM_Q36_d,FNM_Q37_m_1,FNM_Q37_m_2,FNM_Q37_m_3,FNM_Q37_m_4,FNM_Q37_m_5,FNM_Q37_m_6,FNM_Q37_m_7,FNM_Q37_m_8,FNM_Q37_m_9,FNM_Q37_m_955,FNM_Q37_m_933,FNM_Q37_open1,FNM_Q38,FNM_Q1_1,FNM_Q1_2,FNM_Q1_3,FNM_Q1_4,FNM_Q1_5,FNM_Q1_6,FNM_Q1_attn,FNM_Q1_7,FNM_Q1_8,FNM_Q1_9,FNM_Q2_1,FNM_Q2_2,FNM_Q2_3,FNM_Q2_4,FNM_Q2_5,FNM_Q3_1,FNM_Q3_2,FNM_Q3_3,FNM_Q3_4,FNM_Q4_1,FNM_Q4_2,FNM_Q4_3,FNM_Q4_4,FNM_Q4_5,FNM_Q4_6,FNM_Q4_7,FNM_Q4_8,FNM_Q4_9,FNM_Q5_1,FNM_Q5_2,FNM_Q5_3,FNM_Q5_4,FNM_Q5_5,FNM_Q5_6,FNM_Q5_7,FNM_Q6_1,FNM_Q6_2,FNM_Q6_3,FNM_Q6_4,FNM_Q6_5,FNM_Q6_6,FNM_Q6_7,FNM_Q6_8,FNM_Q7_1,FNM_Q7_2,FNM_Q7_3,FNM_Q7_4,FNM_Q7_5,FNM_Q8_1,FNM_Q8_2,FNM_Q8_3,FNM_Q8_4,FNM_Q8_5,FNM_Q8_6,FNM_Q9_1,FNM_Q9_2,FNM_Q9_3,FNM_Q9_4,FNM_Q9_5,FNM_Q10_1,FNM_Q10_2,FNM_Q10_3,FNM_Q10_4,FNM_Q10_5,FNM_Q11_1,FNM_Q11_2,FNM_Q11_3,FNM_Q11_4,FNM_Q11_5,FNM_Q12_1,FNM_Q12_2,FNM_Q12_3,FNM_Q12_4,FNM_Q12_5,FNM_Q13_1,FNM_Q13_2,FNM_Q13_3,FNM_Q13_4,FNM_Q13_5,FNM_Q14_1,FNM_Q14_2,FNM_Q14_3,FNM_Q14_4,FNM_Q14_5,FNM_Q15_1,FNM_Q15_2,FNM_Q15_3,FNM_Q15_4,FNM_Q15_5,FNM_Q16_1,FNM_Q16_2,FNM_Q16_3,FNM_Q16_4,FNM_Q16_5,FNM_Q17_1,FNM_Q17_2,FNM_Q17_3,FNM_Q17_4,FNM_Q17_5,FNM_Q18_1,FNM_Q18_2,FNM_Q18_3,FNM_Q18_4,FNM_Q18_5,FNM_Q19_1,FNM_Q19_2,FNM_Q19_3,FNM_Q19_4,FNM_Q19_5,FNM_Q20_1,FNM_Q20_2,FNM_Q20_3,FNM_Q20_4,FNM_Q20_5,FNM_Q21_1,FNM_Q21_2,FNM_Q21_3,FNM_Q21_4,FNM_Q21_5,FNM_Q22_1,FNM_Q22_2,FNM_Q22_3,FNM_Q22_4,FNM_Q22_5,FNM_Q22_6,FNM_Q23_1,FNM_Q23_2,FNM_Q23_3,FNM_Q40_m_1,FNM_Q40_m_2,FNM_Q40_m_3,FNM_Q40_m_4,FNM_Q40_m_5,FNM_Q40_m_6,FNM_Q40_m_7,FNM_Q40_m_8,FNM_Q40_m_9,FNM_Q40_m_10,FNM_Q40_m_11,FNM_Q40_m_12,FNM_Q40_m_13,FNM_Q40_m_14,FNM_Q40_m_15,FNM_Q40_m_16,FNM_Q40_m_17,FNM_Q40_m_18,FNM_Q40_m_19,FNM_Q40_m_20,FNM_Q40_m_21,FNM_Q40_m_22,FNM_Q40_m_23,FNM_Q40_m_24,FNM_Q40_m_25,FNM_Q40_m_26,FNM_Q40_m_27,FNM_Q40_m_944,FNM_Q41_m_1,FNM_Q41_m_2,FNM_Q41_m_3,FNM_Q41_m_4,...,FNM_Q3_1_recontact,FNM_Q3_2_recontact,FNM_Q3_3_recontact,FNM_Q3_4_recontact,FNM_Q4_1_recontact,FNM_Q4_2_recontact,FNM_Q4_3_recontact,FNM_Q4_4_recontact,FNM_Q4_5_recontact,FNM_Q4_6_recontact,FNM_Q4_7_recontact,FNM_Q4_8_recontact,FNM_Q4_9_recontact,FNM_Q5_1_recontact,FNM_Q5_2_recontact,FNM_Q5_3_recontact,FNM_Q5_4_recontact,FNM_Q5_5_recontact,FNM_Q5_6_recontact,FNM_Q5_7_recontact,FNM_Q6_1_recontact,FNM_Q6_2_recontact,FNM_Q6_3_recontact,FNM_Q6_4_recontact,FNM_Q6_5_recontact,FNM_Q6_6_recontact,FNM_Q6_7_recontact,FNM_Q6_8_recontact,FNM_Q7_1_recontact,FNM_Q7_2_recontact,FNM_Q7_3_recontact,FNM_Q7_4_recontact,FNM_Q7_5_recontact,FNM_Q8_1_recontact,FNM_Q8_2_recontact,FNM_Q8_3_recontact,FNM_Q8_4_recontact,FNM_Q8_5_recontact,FNM_Q8_6_recontact,FNM_Q9_1_recontact,FNM_Q9_2_recontact,FNM_Q9_3_recontact,FNM_Q9_4_recontact,FNM_Q9_5_recontact,FNM_Q10_1_recontact,FNM_Q10_2_recontact,FNM_Q10_3_recontact,FNM_Q10_4_recontact,FNM_Q10_5_recontact,FNM_Q11_1_recontact,FNM_Q11_2_recontact,FNM_Q11_3_recontact,FNM_Q11_4_recontact,FNM_Q11_5_recontact,FNM_Q12_1_recontact,FNM_Q12_2_recontact,FNM_Q12_3_recontact,FNM_Q12_4_recontact,FNM_Q12_5_recontact,FNM_Q13_1_recontact,FNM_Q13_2_recontact,FNM_Q13_3_recontact,FNM_Q13_4_recontact,FNM_Q13_5_recontact,FNM_Q14_1_recontact,FNM_Q14_2_recontact,FNM_Q14_3_recontact,FNM_Q14_4_recontact,FNM_Q14_5_recontact,FNM_Q15_1_recontact,FNM_Q15_2_re

In [105]:
#loads the codesheet file
# wherever the emane column exists, creates a dict variablename:ename
code_book = pd.read_csv('../dylan_github/yougov_codesheet_alignment.tsv', sep='\t')
ename_lut = {vn:en for vn, en in code_book.loc[code_book.ename.notnull(), ['var_name', 'ename']].values}
ename_lut

{'caseid': 'caseid',
 'birthyr': 'birthyr',
 'gender': 'gender',
 'sex': 'sex',
 'educ': 'educ',
 'inputstate': 'inputstate',
 'sample_type': 'sample_type',
 'screener_1': 'screener_1ongoing',
 'screener_2': 'screener_2impact',
 'screener_3_1': 'screener_3depression',
 'screener_3_2': 'screener_4anxiety',
 'screener_3_3': 'screener_5attention',
 'consent': 'consent',
 'FNM_Q24': 'mood_yn',
 'FNM_Q26_d': 'mood_years',
 'FNM_Q28': 'mood_bothered',
 'FNM_Q29': 'anxiety_yn',
 'FNM_Q31_d': 'anxiety_years',
 'FNM_Q33': 'anxiety_bothered',
 'FNM_34': 'attention_yn',
 'FNM_Q36_d': 'attention_years',
 'FNM_Q38': 'attention_bothered',
 'FNM_Q1_1': 'inattention_1',
 'FNM_Q1_2': 'inattention_2',
 'FNM_Q1_3': 'inattention_3',
 'FNM_Q1_4': 'inattention_4',
 'FNM_Q1_5': 'inattention_5',
 'FNM_Q1_6': 'inattention_6',
 'FNM_Q1_7': 'inattention_7',
 'FNM_Q1_8': 'inattention_8',
 'FNM_Q1_9': 'inattention_9',
 'FNM_Q2_1': 'hyperactivity_1',
 'FNM_Q2_2': 'hyperactivity_2',
 'FNM_Q2_3': 'hyperactivity_3',
 

In [106]:
for vn, en in code_book.loc[code_book.ename.notnull(), ['var_name', 'ename']].values:
    ename_lut[vn+'_recontact'] = en+'_recontact'
ename_lut

{'caseid': 'caseid',
 'birthyr': 'birthyr',
 'gender': 'gender',
 'sex': 'sex',
 'educ': 'educ',
 'inputstate': 'inputstate',
 'sample_type': 'sample_type',
 'screener_1': 'screener_1ongoing',
 'screener_2': 'screener_2impact',
 'screener_3_1': 'screener_3depression',
 'screener_3_2': 'screener_4anxiety',
 'screener_3_3': 'screener_5attention',
 'consent': 'consent',
 'FNM_Q24': 'mood_yn',
 'FNM_Q26_d': 'mood_years',
 'FNM_Q28': 'mood_bothered',
 'FNM_Q29': 'anxiety_yn',
 'FNM_Q31_d': 'anxiety_years',
 'FNM_Q33': 'anxiety_bothered',
 'FNM_34': 'attention_yn',
 'FNM_Q36_d': 'attention_years',
 'FNM_Q38': 'attention_bothered',
 'FNM_Q1_1': 'inattention_1',
 'FNM_Q1_2': 'inattention_2',
 'FNM_Q1_3': 'inattention_3',
 'FNM_Q1_4': 'inattention_4',
 'FNM_Q1_5': 'inattention_5',
 'FNM_Q1_6': 'inattention_6',
 'FNM_Q1_7': 'inattention_7',
 'FNM_Q1_8': 'inattention_8',
 'FNM_Q1_9': 'inattention_9',
 'FNM_Q2_1': 'hyperactivity_1',
 'FNM_Q2_2': 'hyperactivity_2',
 'FNM_Q2_3': 'hyperactivity_3',
 

In [107]:
# this groups hitop items by subscales
# marie: add the initial and the recontact scales

tmp = code_book.query('ename.notnull()')
scale_lut = {}
for ss, df in tmp.loc[tmp.ename.str.contains('hitop') & ~tmp.ename.str.contains('today')].groupby('subscale'):
    scale_name = ss.replace(" ", "_").replace("/", "_").replace("-", "_")
    items = df.ename.values
    scale_lut[scale_name] = items
    
 #   scale_name_initial = scale_name+'_initial'
 #   scale_lut[scale_name_initial] = items 
    scale_name_recontact = scale_name+'_recontact'
    items_recontact = []
    for item in items:
        items_recontact.append(item+'_recontact')
    scale_lut[scale_name_recontact] = np.array(items_recontact, dtype=object)
scale_lut

{'anhedonic_depression': array(['hitop157', 'hitop182', 'hitop93', 'hitop84', 'hitop77', 'hitop92',
        'hitop39', 'hitop246', 'hitop230', 'hitop123'], dtype=object),
 'anhedonic_depression_recontact': array(['hitop157_recontact', 'hitop182_recontact', 'hitop93_recontact',
        'hitop84_recontact', 'hitop77_recontact', 'hitop92_recontact',
        'hitop39_recontact', 'hitop246_recontact', 'hitop230_recontact',
        'hitop123_recontact'], dtype=object),
 'anxious_worry': array(['hitop34', 'hitop89', 'hitop265', 'hitop20', 'hitop248',
        'hitop240', 'hitop203'], dtype=object),
 'anxious_worry_recontact': array(['hitop34_recontact', 'hitop89_recontact', 'hitop265_recontact',
        'hitop20_recontact', 'hitop248_recontact', 'hitop240_recontact',
        'hitop203_recontact'], dtype=object),
 'appetite_gain': array(['hitop243', 'hitop141', 'hitop120', 'hitop275'], dtype=object),
 'appetite_gain_recontact': array(['hitop243_recontact', 'hitop141_recontact', 'hitop120_recont

In [108]:
# done working with the codesheet file, now opening the actual data

In [109]:
datapath = '../data/NIMH0007_genpop_num_OUTPUT.csv'
dat = pd.read_csv(datapath, dtype={'caseid':str}, engine='python')
dat.head()

,caseid,weight,sample_type,screener_1,screener_2,screener_3_1,screener_3_2,screener_3_3,consent,FNM_Q24,FNM_Q25_1,FNM_Q25_2,FNM_Q25_3,FNM_Q25_4,FNM_Q25_5,FNM_Q25_6,FNM_Q25_955,FNM_Q25_933,FNM_Q25_open1,FNM_Q26_d,FNM_Q27_1,FNM_Q27_2,FNM_Q27_3,FNM_Q27_4,FNM_Q27_5,FNM_Q27_6,FNM_Q27_7,FNM_Q27_8,FNM_Q27_9,FNM_Q27_10,FNM_Q27_11,FNM_Q27_12,FNM_Q27_13,FNM_Q27_14,FNM_Q27_15,FNM_Q27_16,FNM_Q27_17,FNM_Q27_18,FNM_Q27_19,FNM_Q27_955,FNM_Q27_933,FNM_Q27_open1,FNM_Q28,FNM_Q29,FNM_Q30_m_1,FNM_Q30_m_2,FNM_Q30_m_3,FNM_Q30_m_4,FNM_Q30_m_5,FNM_Q30_m_6,FNM_Q30_m_7,FNM_Q30_m_8,FNM_Q30_m_955,FNM_Q30_m_933,FNM_Q30_open1,FNM_Q31_d,FNM_Q32_1,FNM_Q32_2,FNM_Q32_3,FNM_Q32_4,FNM_Q32_5,FNM_Q32_6,FNM_Q32_7,FNM_Q32_8,FNM_Q32_9,FNM_Q32_10,FNM_Q32_955,FNM_Q32_933,FNM_Q32_open1,FNM_Q33,FNM_34,FNM_Q35_m_1,FNM_Q35_m_2,FNM_Q35_m_3,FNM_Q35_m_933,FNM_Q35_open1,FNM_Q36_d,FNM_Q37_m_1,FNM_Q37_m_2,FNM_Q37_m_3,FNM_Q37_m_4,FNM_Q37_m_5,FNM_Q37_m_6,FNM_Q37_m_7,FNM_Q37_m_8,FNM_Q37_m_9,FNM_Q37_m_955,FNM_Q37_m_933,FNM_Q37_open1,FNM_Q38,FNM_Q1_1,FNM_Q1_2,FNM_Q1_3,FNM_Q1_4,FNM_Q1_5,FNM_Q1_6,FNM_Q1_attn,FNM_Q1_7,FNM_Q1_8,FNM_Q1_9,FNM_Q2_1,FNM_Q2_2,FNM_Q2_3,FNM_Q2_4,FNM_Q2_5,FNM_Q3_1,FNM_Q3_2,FNM_Q3_3,FNM_Q3_4,FNM_Q4_1,FNM_Q4_2,FNM_Q4_3,FNM_Q4_4,FNM_Q4_5,FNM_Q4_6,FNM_Q4_7,FNM_Q4_8,FNM_Q4_9,FNM_Q5_1,FNM_Q5_2,FNM_Q5_3,FNM_Q5_4,FNM_Q5_5,FNM_Q5_6,FNM_Q5_7,FNM_Q6_1,FNM_Q6_2,FNM_Q6_3,FNM_Q6_4,FNM_Q6_5,FNM_Q6_6,FNM_Q6_7,FNM_Q6_8,FNM_Q7_1,FNM_Q7_2,FNM_Q7_3,FNM_Q7_4,FNM_Q7_5,FNM_Q8_1,FNM_Q8_2,FNM_Q8_3,FNM_Q8_4,FNM_Q8_5,FNM_Q8_6,FNM_Q9_1,FNM_Q9_2,FNM_Q9_3,FNM_Q9_4,FNM_Q9_5,FNM_Q10_1,FNM_Q10_2,FNM_Q10_3,FNM_Q10_4,FNM_Q10_5,FNM_Q11_1,FNM_Q11_2,FNM_Q11_3,FNM_Q11_4,FNM_Q11_5,FNM_Q12_1,FNM_Q12_2,FNM_Q12_3,FNM_Q12_4,FNM_Q12_5,FNM_Q13_1,FNM_Q13_2,FNM_Q13_3,FNM_Q13_4,FNM_Q13_5,FNM_Q14_1,FNM_Q14_2,FNM_Q14_3,FNM_Q14_4,FNM_Q14_5,FNM_Q15_1,FNM_Q15_2,FNM_Q15_3,FNM_Q15_4,FNM_Q15_5,FNM_Q16_1,FNM_Q16_2,FNM_Q16_3,FNM_Q16_4,FNM_Q16_5,FNM_Q17_1,FNM_Q17_2,FNM_Q17_3,FNM_Q17_4,FNM_Q17_5,FNM_Q18_1,FNM_Q18_2,FNM_Q18_3,FNM_Q18_4,FNM_Q18_5,FNM_Q19_1,FNM_Q19_2,FNM_Q19_3,FNM_Q19_4,FNM_Q19_5,FNM_Q20_1,FNM_Q20_2,FNM_Q20_3,FNM_Q20_4,FNM_Q20_5,FNM_Q21_1,FNM_Q21_2,FNM_Q21_3,FNM_Q21_4,FNM_Q21_5,FNM_Q22_1,FNM_Q22_2,FNM_Q22_3,FNM_Q22_4,FNM_Q22_5,FNM_Q22_6,FNM_Q23_1,FNM_Q23_2,FNM_Q23_3,FNM_Q40_m_1,FNM_Q40_m_2,FNM_Q40_m_3,FNM_Q40_m_4,FNM_Q40_m_5,FNM_Q40_m_6,FNM_Q40_m_7,FNM_Q40_m_8,FNM_Q40_m_9,FNM_Q40_m_10,FNM_Q40_m_11,FNM_Q40_m_12,FNM_Q40_m_13,FNM_Q40_m_14,FNM_Q40_m_15,FNM_Q40_m_16,FNM_Q40_m_17,FNM_Q40_m_18,FNM_Q40_m_19,FNM_Q40_m_20,FNM_Q40_m_21,FNM_Q40_m_22,FNM_Q40_m_23,FNM_Q40_m_24,FNM_Q40_m_25,FNM_Q40_m_26,FNM_Q40_m_27,FNM_Q40_m_944,FNM_Q41_m_1,FNM_Q41_m_2,FNM_Q41_m_3,FNM_Q41_m_4,...,FNM_Q3_1_recontact,FNM_Q3_2_recontact,FNM_Q3_3_recontact,FNM_Q3_4_recontact,FNM_Q4_1_recontact,FNM_Q4_2_recontact,FNM_Q4_3_recontact,FNM_Q4_4_recontact,FNM_Q4_5_recontact,FNM_Q4_6_recontact,FNM_Q4_7_recontact,FNM_Q4_8_recontact,FNM_Q4_9_recontact,FNM_Q5_1_recontact,FNM_Q5_2_recontact,FNM_Q5_3_recontact,FNM_Q5_4_recontact,FNM_Q5_5_recontact,FNM_Q5_6_recontact,FNM_Q5_7_recontact,FNM_Q6_1_recontact,FNM_Q6_2_recontact,FNM_Q6_3_recontact,FNM_Q6_4_recontact,FNM_Q6_5_recontact,FNM_Q6_6_recontact,FNM_Q6_7_recontact,FNM_Q6_8_recontact,FNM_Q7_1_recontact,FNM_Q7_2_recontact,FNM_Q7_3_recontact,FNM_Q7_4_recontact,FNM_Q7_5_recontact,FNM_Q8_1_recontact,FNM_Q8_2_recontact,FNM_Q8_3_recontact,FNM_Q8_4_recontact,FNM_Q8_5_recontact,FNM_Q8_6_recontact,FNM_Q9_1_recontact,FNM_Q9_2_recontact,FNM_Q9_3_recontact,FNM_Q9_4_recontact,FNM_Q9_5_recontact,FNM_Q10_1_recontact,FNM_Q10_2_recontact,FNM_Q10_3_recontact,FNM_Q10_4_recontact,FNM_Q10_5_recontact,FNM_Q11_1_recontact,FNM_Q11_2_recontact,FNM_Q11_3_recontact,FNM_Q11_4_recontact,FNM_Q11_5_recontact,FNM_Q12_1_recontact,FNM_Q12_2_recontact,FNM_Q12_3_recontact,FNM_Q12_4_recontact,FNM_Q12_5_recontact,FNM_Q13_1_recontact,FNM_Q13_2_recontact,FNM_Q13_3_recontact,FNM_Q13_4_recontact,FNM_Q13_5_recontact,FNM_Q14_1_recontact,FNM_Q14_2_recontact,FNM_Q14_3_recontact,FNM_Q14_4_recontact,FNM_Q14_5_recontact,FNM_Q15_1_recontact,FNM_Q15_2_re

In [110]:
dat.columns

Index(['caseid', 'weight', 'sample_type', 'screener_1', 'screener_2',
       'screener_3_1', 'screener_3_2', 'screener_3_3', 'consent', 'FNM_Q24',
       ...
       'FNM_44_m_31_recontact', 'FNM_44_m_32_recontact',
       'FNM_44_m_33_recontact', 'FNM_44_m_34_recontact',
       'FNM_44_m_35_recontact', 'family_wealth_recontact',
       'WHODAS_H2_recontact', 'WHODAS_H3_recontact', 'starttime_recontact',
       'endtime_recontact'],
      dtype='object', length=728)

In [111]:
# drop the .0 that pandas appends for some reason
dat['caseid'] = dat.caseid.str[:-2]
# i'm not sure what this does
dat = dat.rename(ename_lut, axis=1)

In [112]:
dat_genpop = dat
dat_genpop.head(3)

,caseid,weight,sample_type,screener_1ongoing,screener_2impact,screener_3depression,screener_4anxiety,screener_5attention,consent,mood_yn,FNM_Q25_1,FNM_Q25_2,FNM_Q25_3,FNM_Q25_4,FNM_Q25_5,FNM_Q25_6,FNM_Q25_955,FNM_Q25_933,FNM_Q25_open1,mood_years,FNM_Q27_1,FNM_Q27_2,FNM_Q27_3,FNM_Q27_4,FNM_Q27_5,FNM_Q27_6,FNM_Q27_7,FNM_Q27_8,FNM_Q27_9,FNM_Q27_10,FNM_Q27_11,FNM_Q27_12,FNM_Q27_13,FNM_Q27_14,FNM_Q27_15,FNM_Q27_16,FNM_Q27_17,FNM_Q27_18,FNM_Q27_19,FNM_Q27_955,FNM_Q27_933,FNM_Q27_open1,mood_bothered,anxiety_yn,FNM_Q30_m_1,FNM_Q30_m_2,FNM_Q30_m_3,FNM_Q30_m_4,FNM_Q30_m_5,FNM_Q30_m_6,FNM_Q30_m_7,FNM_Q30_m_8,FNM_Q30_m_955,FNM_Q30_m_933,FNM_Q30_open1,anxiety_years,FNM_Q32_1,FNM_Q32_2,FNM_Q32_3,FNM_Q32_4,FNM_Q32_5,FNM_Q32_6,FNM_Q32_7,FNM_Q32_8,FNM_Q32_9,FNM_Q32_10,FNM_Q32_955,FNM_Q32_933,FNM_Q32_open1,anxiety_bothered,attention_yn,FNM_Q35_m_1,FNM_Q35_m_2,FNM_Q35_m_3,FNM_Q35_m_933,FNM_Q35_open1,attention_years,FNM_Q37_m_1,FNM_Q37_m_2,FNM_Q37_m_3,FNM_Q37_m_4,FNM_Q37_m_5,FNM_Q37_m_6,FNM_Q37_m_7,FNM_Q37_m_8,FNM_Q37_m_9,FNM_Q37_m_955,FNM_Q37_m_933,FNM_Q37_open1,attention_bothered,inattention_1,inattention_2,inattention_3,inattention_4,inattention_5,inattention_6,FNM_Q1_attn,inattention_7,inattention_8,inattention_9,hyperactivity_1,hyperactivity_2,hyperactivity_3,hyperactivity_4,hyperactivity_5,impulsivity_1,impulsivity_2,impulsivity_3,impulsivity_4,sct_1,sct_2,sct_3,sct_4,sct_5,sct_6,sct_7,sct_8,sct_9,gad_1,gad_2,gad_3,gad_4,gad_5,gad_6,gad_7,phq_1,phq_2,phq_3,phq_4,phq_5,phq_6,phq_7,phq_8,hitop157,hitop81,hitop34,hitop54,hitop243,hitop182,hitop69,hitop89,hitop50,check_moderately,hitop129,hitop265,hitop124,hitop231,hitop93,hitop67,hitop245,hitop281,hitop141,hitop40,hitop204,hitop21,hitop236,hitop280,hitop84,hitop120,hitop77,hitop92,hitop258,hitop39,hitop254,hitop215,hitop95,hitop106,hitop283,hitop16,hitop20,hitop189,hitop1,hitop136,hitop246,hitop248,hitop257,hitop114,hitop117,hitop250,hitop200,hitop160,hitop23,hitop165,hitop244,hitop9,hitop142,hitop230,hitop149,hitop247,hitop99,hitop66,hitop240,hitop222,hitop90,hitop113,hitop278,hitop203,hitop159,hitop123,hitop275,hitop268,hitop225,hitop143,hitop151,hitop181,hitop211,hitop17,hitop126,hitop5,hitop261,hitop220,check_notatall,hitop15,hitop72,hitop140,hitop109,hitop197,hitop104,todayinattention_1,todayinattention_2,todayinattention_3,todayinattention_4,todayinattention_5,todayinattention_6,todayinattention_7,todayinattention_8,todayinattention_9,todayhyperactivity_1,todayhyperactivity_2,todayhyperactivity_3,todayhyperactivity_4,todayhyperactivity_5,todayimpulsivity_1,todayimpulsivity_2,todayimpulsivity_3,todayimpulsivity_4,todaysct_1,todaysct_2,todaysct_3,todaysct_4,todaysct_5,todaysct_6,todaysct_7,todaysct_8,todaysct_9,today_na1,todaygad_1,todaygad_2,todaygad_3,todaygad_4,...,impulsivity_1_recontact,impulsivity_2_recontact,impulsivity_3_recontact,impulsivity_4_recontact,sct_1_recontact,sct_2_recontact,sct_3_recontact,sct_4_recontact,sct_5_recontact,sct_6_recontact,sct_7_recontact,sct_8_recontact,sct_9_recontact,gad_1_recontact,gad_2_recontact,gad_3_recontact,gad_4_recontact,gad_5_recontact,gad_6_recontact,gad_7_recontact,phq_1_recontact,phq_2_recontact,phq_3_recontact,phq_4_recontact,phq_5_recontact,phq_6_recontact,phq_7_recontact,phq_8_recontact,hitop157_recontact,hitop81_recontact,hitop34_recontact,hitop54_recontact,hitop243_recontact,hitop182_recontact,hitop69_recontact,hitop89_recontact,hitop50_recontact,check_moderately_recontact,hitop129_recontact,hitop265_recontact,hitop124_recontact,hitop231_recontact,hitop93_recontact,hitop67_recontact,hitop245_recontact,hitop281_recontact,hitop141_recontact,hitop40_recontact,hitop204_recontact,hitop21_recontact,hitop236_recontact,hitop280_recontact,hitop84_recontact,hitop120_recontact,hitop77_recontact,hitop92_recontact,hitop258_recontact,hitop39_recontact,hitop254_recontact,hitop215_recontact,hitop95_recontact,hitop106_recontact,hitop283_recontact,hitop16_recontact,hitop20_recontact,hitop189_recontact,hitop1_recontact,hitop136_recontact,hitop246_reconta

## check how dat dataframes look like

In [113]:
dat_validation.head()

,gender,race,hispanic,current_txt,past_txt,current_med,age,hitop39,hitop77,hitop84,hitop92,hitop93,hitop123,hitop157,hitop182,hitop230,hitop246,hitop20,hitop34,hitop89,hitop203,hitop240,hitop248,hitop265,hitop120,hitop141,hitop243,hitop275,hitop109,hitop280,hitop283,hitop67,hitop142,hitop159,hitop189,hitop5,hitop66,hitop99,hitop181,hitop231,hitop21,hitop90,hitop95,hitop160,hitop254,hitop261,hitop268,hitop15,hitop104,hitop126,hitop211,hitop215,hitop257,hitop40,hitop50,hitop69,hitop81,hitop113,hitop136,hitop151,hitop197,hitop72,hitop140,hitop143,hitop220,hitop16,hitop165,hitop225,hitop247,hitop278,hitop1,hitop17,hitop114,hitop117,hitop124,hitop129,hitop204,hitop222,hitop236,hitop258,hitop9,hitop23,hitop54,hitop106,hitop149,hitop200,hitop244,hitop245,hitop250,hitop281
0,1,5,2,1,1,1,24,4,3,3,3,4,1,3,2,2,2,3,3,2,2,2,1,2,1,2,2,2,2,2,3,3,2,2,2,2,3,2,1,1,3,2,2,1,2,3,1,3,1,3,2,1,2,3,2,2,1,3,2,2,2,3,2,2,2,1,2,1,4,1,3,4,2,3,2,1,3,1,2,1,3,3,3,2,2,2,2,2,1,2
1,1,2,2,2,1,2,32,2,1,1,2,2,1,2,2,2,1,2,3,2,2,1,1,2,1,1,2,1,1,1,1,2,2,1,2,2,1,1,1,1,2,1,1,2,3,2,2,2,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,2,1,3,1,1,1,1,1,1,1,1,1,1,1,2,3,3,3,3,2,1,3,3,3
2,2,5,2,2,2,1,68,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,2,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,4,1,1,1,1,2,1,1,1,1,1,1,3,4,4,4,4,3,4,4,4,4
3,2,5,1,2,2,1,31,1,1,1,2,1,1,4,1,2,1,3,1,2,3,1,1,2,1,1,1,1,1,1,1,2,2,2,2,1,1,1,1,1,1,1,1,4,4,4,4,1,3,1,1,3,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,3,1,2,1,1,4,4,1,1,4,3,4,3,3,2,3,3,3,1
4,1,2,2,2,1,2,32,2,3,4,3,2,3,3,3,1,2,4,3,3,4,3,4,4,1,1,1,1,2,2,1,4,2,4,4,2,1,1,1,1,4,4,4,2,1,2,2,1,1,1,1,1,1,4,2,1,4,4,4,3,1,2,4,4,4,1,1,1,1,3,1,2,1,4,2,1,2,2,2,1,2,2,2,1,2,3,2,2,2,2


In [114]:
dat_genpop.head()

,caseid,weight,sample_type,screener_1ongoing,screener_2impact,screener_3depression,screener_4anxiety,screener_5attention,consent,mood_yn,FNM_Q25_1,FNM_Q25_2,FNM_Q25_3,FNM_Q25_4,FNM_Q25_5,FNM_Q25_6,FNM_Q25_955,FNM_Q25_933,FNM_Q25_open1,mood_years,FNM_Q27_1,FNM_Q27_2,FNM_Q27_3,FNM_Q27_4,FNM_Q27_5,FNM_Q27_6,FNM_Q27_7,FNM_Q27_8,FNM_Q27_9,FNM_Q27_10,FNM_Q27_11,FNM_Q27_12,FNM_Q27_13,FNM_Q27_14,FNM_Q27_15,FNM_Q27_16,FNM_Q27_17,FNM_Q27_18,FNM_Q27_19,FNM_Q27_955,FNM_Q27_933,FNM_Q27_open1,mood_bothered,anxiety_yn,FNM_Q30_m_1,FNM_Q30_m_2,FNM_Q30_m_3,FNM_Q30_m_4,FNM_Q30_m_5,FNM_Q30_m_6,FNM_Q30_m_7,FNM_Q30_m_8,FNM_Q30_m_955,FNM_Q30_m_933,FNM_Q30_open1,anxiety_years,FNM_Q32_1,FNM_Q32_2,FNM_Q32_3,FNM_Q32_4,FNM_Q32_5,FNM_Q32_6,FNM_Q32_7,FNM_Q32_8,FNM_Q32_9,FNM_Q32_10,FNM_Q32_955,FNM_Q32_933,FNM_Q32_open1,anxiety_bothered,attention_yn,FNM_Q35_m_1,FNM_Q35_m_2,FNM_Q35_m_3,FNM_Q35_m_933,FNM_Q35_open1,attention_years,FNM_Q37_m_1,FNM_Q37_m_2,FNM_Q37_m_3,FNM_Q37_m_4,FNM_Q37_m_5,FNM_Q37_m_6,FNM_Q37_m_7,FNM_Q37_m_8,FNM_Q37_m_9,FNM_Q37_m_955,FNM_Q37_m_933,FNM_Q37_open1,attention_bothered,inattention_1,inattention_2,inattention_3,inattention_4,inattention_5,inattention_6,FNM_Q1_attn,inattention_7,inattention_8,inattention_9,hyperactivity_1,hyperactivity_2,hyperactivity_3,hyperactivity_4,hyperactivity_5,impulsivity_1,impulsivity_2,impulsivity_3,impulsivity_4,sct_1,sct_2,sct_3,sct_4,sct_5,sct_6,sct_7,sct_8,sct_9,gad_1,gad_2,gad_3,gad_4,gad_5,gad_6,gad_7,phq_1,phq_2,phq_3,phq_4,phq_5,phq_6,phq_7,phq_8,hitop157,hitop81,hitop34,hitop54,hitop243,hitop182,hitop69,hitop89,hitop50,check_moderately,hitop129,hitop265,hitop124,hitop231,hitop93,hitop67,hitop245,hitop281,hitop141,hitop40,hitop204,hitop21,hitop236,hitop280,hitop84,hitop120,hitop77,hitop92,hitop258,hitop39,hitop254,hitop215,hitop95,hitop106,hitop283,hitop16,hitop20,hitop189,hitop1,hitop136,hitop246,hitop248,hitop257,hitop114,hitop117,hitop250,hitop200,hitop160,hitop23,hitop165,hitop244,hitop9,hitop142,hitop230,hitop149,hitop247,hitop99,hitop66,hitop240,hitop222,hitop90,hitop113,hitop278,hitop203,hitop159,hitop123,hitop275,hitop268,hitop225,hitop143,hitop151,hitop181,hitop211,hitop17,hitop126,hitop5,hitop261,hitop220,check_notatall,hitop15,hitop72,hitop140,hitop109,hitop197,hitop104,todayinattention_1,todayinattention_2,todayinattention_3,todayinattention_4,todayinattention_5,todayinattention_6,todayinattention_7,todayinattention_8,todayinattention_9,todayhyperactivity_1,todayhyperactivity_2,todayhyperactivity_3,todayhyperactivity_4,todayhyperactivity_5,todayimpulsivity_1,todayimpulsivity_2,todayimpulsivity_3,todayimpulsivity_4,todaysct_1,todaysct_2,todaysct_3,todaysct_4,todaysct_5,todaysct_6,todaysct_7,todaysct_8,todaysct_9,today_na1,todaygad_1,todaygad_2,todaygad_3,todaygad_4,...,impulsivity_1_recontact,impulsivity_2_recontact,impulsivity_3_recontact,impulsivity_4_recontact,sct_1_recontact,sct_2_recontact,sct_3_recontact,sct_4_recontact,sct_5_recontact,sct_6_recontact,sct_7_recontact,sct_8_recontact,sct_9_recontact,gad_1_recontact,gad_2_recontact,gad_3_recontact,gad_4_recontact,gad_5_recontact,gad_6_recontact,gad_7_recontact,phq_1_recontact,phq_2_recontact,phq_3_recontact,phq_4_recontact,phq_5_recontact,phq_6_recontact,phq_7_recontact,phq_8_recontact,hitop157_recontact,hitop81_recontact,hitop34_recontact,hitop54_recontact,hitop243_recontact,hitop182_recontact,hitop69_recontact,hitop89_recontact,hitop50_recontact,check_moderately_recontact,hitop129_recontact,hitop265_recontact,hitop124_recontact,hitop231_recontact,hitop93_recontact,hitop67_recontact,hitop245_recontact,hitop281_recontact,hitop141_recontact,hitop40_recontact,hitop204_recontact,hitop21_recontact,hitop236_recontact,hitop280_recontact,hitop84_recontact,hitop120_recontact,hitop77_recontact,hitop92_recontact,hitop258_recontact,hitop39_recontact,hitop254_recontact,hitop215_recontact,hitop95_recontact,hitop106_recontact,hitop283_recontact,hitop16_recontact,hitop20_recontact,hitop189_recontact,hitop1_recontact,hitop136_recontact,hitop246_reconta

In [115]:
for c in dat_validation.columns:
    print(c)

gender
race
hispanic
current_txt
past_txt
current_med
age
hitop39
hitop77
hitop84
hitop92
hitop93
hitop123
hitop157
hitop182
hitop230
hitop246
hitop20
hitop34
hitop89
hitop203
hitop240
hitop248
hitop265
hitop120
hitop141
hitop243
hitop275
hitop109
hitop280
hitop283
hitop67
hitop142
hitop159
hitop189
hitop5
hitop66
hitop99
hitop181
hitop231
hitop21
hitop90
hitop95
hitop160
hitop254
hitop261
hitop268
hitop15
hitop104
hitop126
hitop211
hitop215
hitop257
hitop40
hitop50
hitop69
hitop81
hitop113
hitop136
hitop151
hitop197
hitop72
hitop140
hitop143
hitop220
hitop16
hitop165
hitop225
hitop247
hitop278
hitop1
hitop17
hitop114
hitop117
hitop124
hitop129
hitop204
hitop222
hitop236
hitop258
hitop9
hitop23
hitop54
hitop106
hitop149
hitop200
hitop244
hitop245
hitop250
hitop281


In [116]:
for c in dat_genpop.columns:
    print(c)

caseid
weight
sample_type
screener_1ongoing
screener_2impact
screener_3depression
screener_4anxiety
screener_5attention
consent
mood_yn
FNM_Q25_1
FNM_Q25_2
FNM_Q25_3
FNM_Q25_4
FNM_Q25_5
FNM_Q25_6
FNM_Q25_955
FNM_Q25_933
FNM_Q25_open1
mood_years
FNM_Q27_1
FNM_Q27_2
FNM_Q27_3
FNM_Q27_4
FNM_Q27_5
FNM_Q27_6
FNM_Q27_7
FNM_Q27_8
FNM_Q27_9
FNM_Q27_10
FNM_Q27_11
FNM_Q27_12
FNM_Q27_13
FNM_Q27_14
FNM_Q27_15
FNM_Q27_16
FNM_Q27_17
FNM_Q27_18
FNM_Q27_19
FNM_Q27_955
FNM_Q27_933
FNM_Q27_open1
mood_bothered
anxiety_yn
FNM_Q30_m_1
FNM_Q30_m_2
FNM_Q30_m_3
FNM_Q30_m_4
FNM_Q30_m_5
FNM_Q30_m_6
FNM_Q30_m_7
FNM_Q30_m_8
FNM_Q30_m_955
FNM_Q30_m_933
FNM_Q30_open1
anxiety_years
FNM_Q32_1
FNM_Q32_2
FNM_Q32_3
FNM_Q32_4
FNM_Q32_5
FNM_Q32_6
FNM_Q32_7
FNM_Q32_8
FNM_Q32_9
FNM_Q32_10
FNM_Q32_955
FNM_Q32_933
FNM_Q32_open1
anxiety_bothered
attention_yn
FNM_Q35_m_1
FNM_Q35_m_2
FNM_Q35_m_3
FNM_Q35_m_933
FNM_Q35_open1
attention_years
FNM_Q37_m_1
FNM_Q37_m_2
FNM_Q37_m_3
FNM_Q37_m_4
FNM_Q37_m_5
FNM_Q37_m_6
FNM_Q37_m_7
FNM_Q37

In [117]:
# do hitop columns match?
for c in dat_validation.columns:
    if 'hitop' in c:
        if c in dat_genpop.columns:
            print(c)
            print('matched in both dataframes')
        else:
            print(c)
            print('only in dat_validation')

hitop39
matched in both dataframes
hitop77
matched in both dataframes
hitop84
matched in both dataframes
hitop92
matched in both dataframes
hitop93
matched in both dataframes
hitop123
matched in both dataframes
hitop157
matched in both dataframes
hitop182
matched in both dataframes
hitop230
matched in both dataframes
hitop246
matched in both dataframes
hitop20
matched in both dataframes
hitop34
matched in both dataframes
hitop89
matched in both dataframes
hitop203
matched in both dataframes
hitop240
matched in both dataframes
hitop248
matched in both dataframes
hitop265
matched in both dataframes
hitop120
matched in both dataframes
hitop141
matched in both dataframes
hitop243
matched in both dataframes
hitop275
matched in both dataframes
hitop109
matched in both dataframes
hitop280
matched in both dataframes
hitop283
matched in both dataframes
hitop67
matched in both dataframes
hitop142
matched in both dataframes
hitop159
matched in both dataframes
hitop189
matched in both dataframes
h

In [118]:
# do hitop columns match?
for c in dat_genpop.columns:
    if 'hitop' in c:
        if c in dat_validation.columns:
            print(c)
            print('matched in both dataframes')
        else:
            print(c)
            print('only in dat_genpop')

hitop157
matched in both dataframes
hitop81
matched in both dataframes
hitop34
matched in both dataframes
hitop54
matched in both dataframes
hitop243
matched in both dataframes
hitop182
matched in both dataframes
hitop69
matched in both dataframes
hitop89
matched in both dataframes
hitop50
matched in both dataframes
hitop129
matched in both dataframes
hitop265
matched in both dataframes
hitop124
matched in both dataframes
hitop231
matched in both dataframes
hitop93
matched in both dataframes
hitop67
matched in both dataframes
hitop245
matched in both dataframes
hitop281
matched in both dataframes
hitop141
matched in both dataframes
hitop40
matched in both dataframes
hitop204
matched in both dataframes
hitop21
matched in both dataframes
hitop236
matched in both dataframes
hitop280
matched in both dataframes
hitop84
matched in both dataframes
hitop120
matched in both dataframes
hitop77
matched in both dataframes
hitop92
matched in both dataframes
hitop258
matched in both dataframes
hitop

## they match, great

In [119]:
## now keep only hitop columns; add a column that indicates group; stack the dfs

In [120]:
hitop_cols = []
for c in dat_validation.columns:
    if 'hitop' in c:
        hitop_cols.append(c)
hitop_cols

['hitop39',
 'hitop77',
 'hitop84',
 'hitop92',
 'hitop93',
 'hitop123',
 'hitop157',
 'hitop182',
 'hitop230',
 'hitop246',
 'hitop20',
 'hitop34',
 'hitop89',
 'hitop203',
 'hitop240',
 'hitop248',
 'hitop265',
 'hitop120',
 'hitop141',
 'hitop243',
 'hitop275',
 'hitop109',
 'hitop280',
 'hitop283',
 'hitop67',
 'hitop142',
 'hitop159',
 'hitop189',
 'hitop5',
 'hitop66',
 'hitop99',
 'hitop181',
 'hitop231',
 'hitop21',
 'hitop90',
 'hitop95',
 'hitop160',
 'hitop254',
 'hitop261',
 'hitop268',
 'hitop15',
 'hitop104',
 'hitop126',
 'hitop211',
 'hitop215',
 'hitop257',
 'hitop40',
 'hitop50',
 'hitop69',
 'hitop81',
 'hitop113',
 'hitop136',
 'hitop151',
 'hitop197',
 'hitop72',
 'hitop140',
 'hitop143',
 'hitop220',
 'hitop16',
 'hitop165',
 'hitop225',
 'hitop247',
 'hitop278',
 'hitop1',
 'hitop17',
 'hitop114',
 'hitop117',
 'hitop124',
 'hitop129',
 'hitop204',
 'hitop222',
 'hitop236',
 'hitop258',
 'hitop9',
 'hitop23',
 'hitop54',
 'hitop106',
 'hitop149',
 'hitop200',
 'h

In [121]:
data_validation = dat_validation[hitop_cols]
data_validation.head(3)
data_validation.shape

(496, 83)

In [122]:
data_validation = data_validation.assign(whichdata=['validation']*496)

In [123]:
data_validation.head()

,hitop39,hitop77,hitop84,hitop92,hitop93,hitop123,hitop157,hitop182,hitop230,hitop246,hitop20,hitop34,hitop89,hitop203,hitop240,hitop248,hitop265,hitop120,hitop141,hitop243,hitop275,hitop109,hitop280,hitop283,hitop67,hitop142,hitop159,hitop189,hitop5,hitop66,hitop99,hitop181,hitop231,hitop21,hitop90,hitop95,hitop160,hitop254,hitop261,hitop268,hitop15,hitop104,hitop126,hitop211,hitop215,hitop257,hitop40,hitop50,hitop69,hitop81,hitop113,hitop136,hitop151,hitop197,hitop72,hitop140,hitop143,hitop220,hitop16,hitop165,hitop225,hitop247,hitop278,hitop1,hitop17,hitop114,hitop117,hitop124,hitop129,hitop204,hitop222,hitop236,hitop258,hitop9,hitop23,hitop54,hitop106,hitop149,hitop200,hitop244,hitop245,hitop250,hitop281,whichdata
0,4,3,3,3,4,1,3,2,2,2,3,3,2,2,2,1,2,1,2,2,2,2,2,3,3,2,2,2,2,3,2,1,1,3,2,2,1,2,3,1,3,1,3,2,1,2,3,2,2,1,3,2,2,2,3,2,2,2,1,2,1,4,1,3,4,2,3,2,1,3,1,2,1,3,3,3,2,2,2,2,2,1,2,validation
1,2,1,1,2,2,1,2,2,2,1,2,3,2,2,1,1,2,1,1,2,1,1,1,1,2,2,1,2,2,1,1,1,1,2,1,1,2,3,2,2,2,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,2,1,3,1,1,1,1,1,1,1,1,1,1,1,2,3,3,3,3,2,1,3,3,3,validation
2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,2,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,4,1,1,1,1,2,1,1,1,1,1,1,3,4,4,4,4,3,4,4,4,4,validation
3,1,1,1,2,1,1,4,1,2,1,3,1,2,3,1,1,2,1,1,1,1,1,1,1,2,2,2,2,1,1,1,1,1,1,1,1,4,4,4,4,1,3,1,1,3,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,3,1,2,1,1,4,4,1,1,4,3,4,3,3,2,3,3,3,1,validation
4,2,3,4,3,2,3,3,3,1,2,4,3,3,4,3,4,4,1,1,1,1,2,2,1,4,2,4,4,2,1,1,1,1,4,4,4,2,1,2,2,1,1,1,1,1,1,4,2,1,4,4,4,3,1,2,4,4,4,1,1,1,1,3,1,2,1,4,2,1,2,2,2,1,2,2,2,1,2,3,2,2,2,2,validation


In [124]:
data_genpop = dat_genpop[hitop_cols]
data_genpop.shape

(500, 83)

In [125]:
data_genpop = data_genpop.assign(whichdata=['genpop']*500)
data_genpop.head()

,hitop39,hitop77,hitop84,hitop92,hitop93,hitop123,hitop157,hitop182,hitop230,hitop246,hitop20,hitop34,hitop89,hitop203,hitop240,hitop248,hitop265,hitop120,hitop141,hitop243,hitop275,hitop109,hitop280,hitop283,hitop67,hitop142,hitop159,hitop189,hitop5,hitop66,hitop99,hitop181,hitop231,hitop21,hitop90,hitop95,hitop160,hitop254,hitop261,hitop268,hitop15,hitop104,hitop126,hitop211,hitop215,hitop257,hitop40,hitop50,hitop69,hitop81,hitop113,hitop136,hitop151,hitop197,hitop72,hitop140,hitop143,hitop220,hitop16,hitop165,hitop225,hitop247,hitop278,hitop1,hitop17,hitop114,hitop117,hitop124,hitop129,hitop204,hitop222,hitop236,hitop258,hitop9,hitop23,hitop54,hitop106,hitop149,hitop200,hitop244,hitop245,hitop250,hitop281,whichdata
0,1,1,1,1,1,1,2,2,1,1,2,3,2,4,2,2,3,1,3,3,2,1,1,1,2,2,2,4,1,1,1,1,1,2,2,2,1,1,2,1,1,1,1,1,1,1,1,1,2,2,1,2,2,1,1,2,2,1,1,1,1,1,1,2,1,1,2,1,2,2,1,2,1,4,4,4,3,4,3,4,4,4,4,genpop
1,1,4,1,2,2,1,2,3,2,2,4,4,3,1,2,2,4,2,4,1,2,2,2,1,3,2,3,2,2,3,2,2,1,2,2,2,3,2,2,1,3,3,3,4,1,4,2,1,1,1,2,1,2,1,1,2,2,2,3,4,2,4,2,3,2,2,2,3,2,4,4,3,2,4,4,4,4,4,4,4,4,1,4,genpop
2,2,3,3,2,3,3,2,2,3,2,2,3,3,3,3,3,2,2,2,3,3,2,3,4,2,3,2,2,2,2,2,2,2,2,3,2,3,2,2,2,3,1,2,3,3,2,3,2,2,2,3,1,2,1,2,3,2,4,3,4,3,2,2,3,3,2,4,4,2,2,2,3,3,2,2,3,4,2,2,3,2,3,3,genpop
3,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,2,2,2,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,2,2,2,4,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,4,1,2,1,1,2,1,1,1,1,1,1,1,1,3,3,3,1,3,1,4,3,1,1,genpop
4,4,3,2,2,3,1,4,4,1,2,2,2,1,3,1,3,1,4,4,4,4,1,1,1,3,4,3,3,1,1,3,1,1,2,2,2,4,4,4,2,2,3,2,2,2,4,3,2,4,1,1,1,3,1,2,2,2,2,1,1,2,2,1,2,2,2,2,2,3,2,1,3,2,2,2,4,1,1,3,1,2,1,1,genpop


In [126]:
mydata = pd.concat([data_validation, data_genpop])
mydata.shape

(996, 84)

In [143]:
mydata.head()

,hitop39,hitop77,hitop84,hitop92,hitop93,hitop123,hitop157,hitop182,hitop230,hitop246,hitop20,hitop34,hitop89,hitop203,hitop240,hitop248,hitop265,hitop120,hitop141,hitop243,hitop275,hitop109,hitop280,hitop283,hitop67,hitop142,hitop159,hitop189,hitop5,hitop66,hitop99,hitop181,hitop231,hitop21,hitop90,hitop95,hitop160,hitop254,hitop261,hitop268,hitop15,hitop104,hitop126,hitop211,hitop215,hitop257,hitop40,hitop50,hitop69,hitop81,hitop113,hitop136,hitop151,hitop197,hitop72,hitop140,hitop143,hitop220,hitop16,hitop165,hitop225,hitop247,hitop278,hitop1,hitop17,hitop114,hitop117,hitop124,hitop129,hitop204,hitop222,hitop236,hitop258,hitop9,hitop23,hitop54,hitop106,hitop149,hitop200,hitop244,hitop245,hitop250,hitop281,whichdata
0,4,3,3,3,4,1,3,2,2,2,3,3,2,2,2,1,2,1,2,2,2,2,2,3,3,2,2,2,2,3,2,1,1,3,2,2,1,2,3,1,3,1,3,2,1,2,3,2,2,1,3,2,2,2,3,2,2,2,1,2,1,4,1,3,4,2,3,2,1,3,1,2,1,3,3,3,2,2,2,2,2,1,2,validation
1,2,1,1,2,2,1,2,2,2,1,2,3,2,2,1,1,2,1,1,2,1,1,1,1,2,2,1,2,2,1,1,1,1,2,1,1,2,3,2,2,2,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,2,1,3,1,1,1,1,1,1,1,1,1,1,1,2,3,3,3,3,2,1,3,3,3,validation
2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,2,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,4,1,1,1,1,2,1,1,1,1,1,1,3,4,4,4,4,3,4,4,4,4,validation
3,1,1,1,2,1,1,4,1,2,1,3,1,2,3,1,1,2,1,1,1,1,1,1,1,2,2,2,2,1,1,1,1,1,1,1,1,4,4,4,4,1,3,1,1,3,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,3,1,2,1,1,4,4,1,1,4,3,4,3,3,2,3,3,3,1,validation
4,2,3,4,3,2,3,3,3,1,2,4,3,3,4,3,4,4,1,1,1,1,2,2,1,4,2,4,4,2,1,1,1,1,4,4,4,2,1,2,2,1,1,1,1,1,1,4,2,1,4,4,4,3,1,2,4,4,4,1,1,1,1,3,1,2,1,4,2,1,2,2,2,1,2,2,2,1,2,3,2,2,2,2,validation


# Now CFA

In [128]:
null_mdl_strs = {}
mdl_strs = {}
for ss, df in item_lookup.groupby('subscale'):
    cln_ss = ss.replace(" ", "_").replace("/", "_").replace("-", "_")
    null_mdl_str = []
    psis = []
    taus = []
    items = []
    for itemn, item in enumerate(df.sort_values('id_ht').htid.values):
        psis.append(f'{item} ~~ c(psi{itemn+1}, psi{itemn+1})*{item}')
        taus.append(f'{item} ~ c(tau{itemn+1}, tau{itemn+1})*{item}')
        items.append(item)
    null_mdl_str = null_mdl_str + psis + taus
    null_mdl_strs[cln_ss] = null_mdl_str
    mdl_strs[cln_ss] = f'{cln_ss} =~' + ' + '.join(items)

In [158]:
# I don't understand how this null works, and I'm not sure where to look for it. Also, we end up not using it. Keeping it in this code but not going back to it later

In [160]:
mdl_strs

{'anhedonic_depression': 'anhedonic_depression =~hitop39 + hitop77 + hitop84 + hitop92 + hitop93 + hitop123 + hitop157 + hitop182 + hitop230 + hitop246',
 'anxious_worry': 'anxious_worry =~hitop20 + hitop34 + hitop89 + hitop203 + hitop248 + hitop265',
 'appetite_gain': 'appetite_gain =~hitop120 + hitop141 + hitop243 + hitop275',
 'appetite_loss': 'appetite_loss =~hitop109 + hitop280 + hitop283',
 'cognitive_problems': 'cognitive_problems =~hitop142 + hitop159 + hitop189',
 'hyposomnia': 'hyposomnia =~hitop5 + hitop66 + hitop99 + hitop181 + hitop231',
 'indecisiveness': 'indecisiveness =~hitop21 + hitop90 + hitop95',
 'insomnia': 'insomnia =~hitop160 + hitop254 + hitop261 + hitop268',
 'panic': 'panic =~hitop15 + hitop104 + hitop126 + hitop211 + hitop215 + hitop257',
 'separation_insecurity': 'separation_insecurity =~hitop40 + hitop50 + hitop69 + hitop81 + hitop113 + hitop136 + hitop151 + hitop197',
 'shame_guilt': 'shame_guilt =~hitop72 + hitop140 + hitop143 + hitop220',
 'situational_

In [161]:
# measures we can't work with because too few degrees of freedom:
# - 'appetite_loss': 'appetite_loss =~hitop109 + hitop280 + hitop283'
# - 'cognitive_problems': 'cognitive_problems =~hitop142 + hitop159 + hitop189',
# - 'indecisiveness': 'indecisiveness =~hitop21 + hitop90 + hitop95'

# here's the dictionary with items that *will* work
mdl_strs_new = {'anhedonic_depression': 'anhedonic_depression =~hitop39 + hitop77 + hitop84 + hitop92 + hitop93 + hitop123 + hitop157 + hitop182 + hitop230 + hitop246',
 'anxious_worry': 'anxious_worry =~hitop20 + hitop34 + hitop89 + hitop203 + hitop248 + hitop265',
 'appetite_gain': 'appetite_gain =~hitop120 + hitop141 + hitop243 + hitop275',
 'hyposomnia': 'hyposomnia =~hitop5 + hitop66 + hitop99 + hitop181 + hitop231',
 'insomnia': 'insomnia =~hitop160 + hitop254 + hitop261 + hitop268',
 'panic': 'panic =~hitop15 + hitop104 + hitop126 + hitop211 + hitop215 + hitop257',
 'separation_insecurity': 'separation_insecurity =~hitop40 + hitop50 + hitop69 + hitop81 + hitop113 + hitop136 + hitop151 + hitop197',
 'shame_guilt': 'shame_guilt =~hitop72 + hitop140 + hitop143 + hitop220',
 'situational_phobia': 'situational_phobia =~hitop16 + hitop165 + hitop225 + hitop247 + hitop278',
 'social_anxiety': 'social_anxiety =~hitop1 + hitop17 + hitop114 + hitop117 + hitop124 + hitop129 + hitop204 + hitop222 + hitop236 + hitop258',
 'well_being': 'well_being =~hitop9 + hitop23 + hitop54 + hitop106 + hitop149 + hitop200 + hitop244 + hitop245 + hitop250 + hitop281'}

In [ ]:
# to load the data in R, let's save it as csv and then load it to R
# there may be a more elegant day to do this, but I don't know it...
mydata.to_csv('../data/CFA/mydata.csv')
#rread_csv = ro.r("read.csv")
ro.r("rdata <- read.csv('../data/CFA/mydata.csv', header=TRUE)")
rdata

In [ ]:
lavaan = importr('lavaan')
semtools = importr('semTools')

In [ ]:
num_iter = 100

for cln_ss, mdl in mdl_strs_new.items():
    print("================")
    print("================")
    print(cln_ss.upper())
    print("================")
    print("================")
    ro.globalenv[cln_ss+'_mdl'] = mdl_strs_new[cln_ss]
    fit_config = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = "whichdata", estimator = "WLSMV")')
    ro.globalenv['fit_config'] = fit_config
    ro.r('print(summary(fit_config, fit.measures=TRUE))')
    print("================")
    out_config = semtools.permuteMeasEq(nPermute=num_iter, # 100 for now, 1000 for the "real" run
                                        con=fit_config, # In the case of testing configural invariance when modelType = "mgcfa", con is the configural model (implicitly, the unconstrained model is the saturated model, so use the defaults uncon = NULL and param = NULL).
                                        parallelType="multicore", ncpus=4)
    _ = print_summary(out_config) 
    print("================")
    fit_metric = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = "whichdata", estimator = "WLSMV", group.equal="loadings")')
    ro.globalenv['fit_metric'] = fit_metric
    ro.r('print(summary(fit_metric, fit.measures=TRUE))')
    print("================")
    out_metric = semtools.permuteMeasEq(nPermute=num_iter, 
                                    uncon=fit_config, 
                                    con=fit_metric, 
                                    param="loadings",
                                    parallelType="multicore", ncpus=4)
    _ = print_summary(out_metric)
    print("================")
    fit_scalar = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = "whichdata", estimator = "WLSMV", group.equal=c("loadings", "intercepts"))')
    ro.globalenv['fit_scalar'] = fit_scalar
    ro.r('print(summary(fit_scalar, fit.measures=TRUE))')
    print("================")
    out_scalar = semtools.permuteMeasEq(nPermute=num_iter, 
                                    uncon=fit_metric, 
                                    con=fit_scalar, 
                                    param=ro.StrVector(["loadings","intercepts"]),
                                    parallelType="multicore", 
                                    ncpus=4)
    _ = print_summary(out_scalar)
    print("================")
    fit_strict = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = "whichdata", estimator = "WLSMV", group.equal=c("loadings", "intercepts", "residuals"))')
    ro.globalenv['fit_strict'] = fit_strict
    ro.r('print(summary(fit_strict, fit.measures=TRUE))')
    out_strict = semtools.permuteMeasEq(nPermute=num_iter, 
                                    uncon=fit_scalar, 
                                    con=fit_strict, 
                                    param=ro.StrVector(["loadings","intercepts", "residuals"]),
                                    parallelType="multicore", 
                                    ncpus=4)
    _ = print_summary(out_strict)
print('DONE')

ANHEDONIC_DEPRESSION
lavaan 0.6.15 ended normally after 34 iterations

  Estimator                                       DWLS
  Optimization method                           NLMINB
  Number of model parameters                        60

  Number of observations per group:                   
    validation                                     496
    genpop                                         500

Model Test User Model:
                                              Standard      Scaled
  Test Statistic                                96.755     284.297
  Degrees of freedom                                70          70
  P-value (Chi-square)                           0.019       0.000
  Scaling correction factor                                  0.358
  Shift parameter                                           13.837
    simple second-order correction                                
  Test statistic for each group:
    validation                                  72.965     210.852
    g

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




Omnibus p value based on parametric chi-squared difference test:

Chisq diff    Df diff Pr(>Chisq) 
   284.297     70.000      0.000 


Omnibus p values based on nonparametric permutation method: 

      AFI.Difference p.value
chisq         96.755       0
Omnibus p value based on parametric chi-squared difference test:

Chisq diff    Df diff Pr(>Chisq) 
   284.297     70.000      0.000 


Omnibus p values based on nonparametric permutation method: 

      AFI.Difference p.value
chisq         96.755       0
lavaan 0.6.15 ended normally after 28 iterations

  Estimator                                       DWLS
  Optimization method                           NLMINB
  Number of model parameters                        60
  Number of equality constraints                     9

  Number of observations per group:                   
    validation                                     496
    genpop                                         500

Model Test User Model:
                            

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




Omnibus p value based on parametric chi-squared difference test:

Chisq diff    Df diff Pr(>Chisq) 
    18.373      9.000      0.031 


Omnibus p values based on nonparametric permutation method: 

      AFI.Difference p.value
chisq         35.793    0.07


 Modification indices for equality constrained parameter estimates,
 with unadjusted 'p.value' based on chi-squared distribution and
 adjusted 'tukey.p.value' based on permutation distribution of the
 maximum modification index per iteration: 

    lhs op   rhs     X2 df p.value tukey.p.value flag
1  .p2. == .p34.  0.035  1   0.852          1.00     
2  .p3. == .p35.  1.101  1   0.294          0.99     
3  .p4. == .p36.  0.110  1   0.740          1.00     
4  .p5. == .p37. 15.274  1   0.000          0.06     
5  .p6. == .p38.  0.514  1   0.474          1.00     
6  .p7. == .p39.  0.420  1   0.517          1.00     
7  .p8. == .p40.  2.311  1   0.128          0.98     
8  .p9. == .p41.  1.628  1   0.202          0.99     
9 .p10. == 

...

Omnibus p value based on parametric chi-squared difference test:

Chisq diff    Df diff Pr(>Chisq) 
    92.046     10.000      0.000 


Omnibus p values based on nonparametric permutation method: 

      AFI.Difference p.value
chisq         63.951       0


 Modification indices for equality constrained parameter estimates,
 with unadjusted 'p.value' based on chi-squared distribution and
 adjusted 'tukey.p.value' based on permutation distribution of the
 maximum modification index per iteration: 

     lhs op   rhs     X2 df p.value tukey.p.value flag                parameter
1   .p2. == .p34.  0.529  1   0.467          1.00       social_anxiety=~hitop17
2   .p3. == .p35. 20.272  1   0.000          0.07      social_anxiety=~hitop114
3   .p4. == .p36. 18.045  1   0.000          0.07      social_anxiety=~hitop117
4   .p5. == .p37.  0.562  1   0.454          1.00      social_anxiety=~hitop124
5   .p6. == .p38. 14.114  1   0.000          0.13      social_anxiety=~hitop129
6   .p7. == .p39.

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




Omnibus p value based on parametric chi-squared difference test:

Chisq diff    Df diff Pr(>Chisq) 
   232.728     70.000      0.000 


Omnibus p values based on nonparametric permutation method: 

      AFI.Difference p.value
chisq         73.512    0.02
Omnibus p value based on parametric chi-squared difference test:

Chisq diff    Df diff Pr(>Chisq) 
   232.728     70.000      0.000 


Omnibus p values based on nonparametric permutation method: 

      AFI.Difference p.value
chisq         73.512    0.02
lavaan 0.6.15 ended normally after 31 iterations

  Estimator                                       DWLS
  Optimization method                           NLMINB
  Number of model parameters                        60
  Number of equality constraints                     9

  Number of observations per group:                   
    validation                                     496
    genpop                                         500

Model Test User Model:
                            

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




Omnibus p value based on parametric chi-squared difference test:

Chisq diff    Df diff Pr(>Chisq) 
    37.192      9.000      0.000 


Omnibus p values based on nonparametric permutation method: 

      AFI.Difference p.value
chisq         76.728       0


 Modification indices for equality constrained parameter estimates,
 with unadjusted 'p.value' based on chi-squared distribution and
 adjusted 'tukey.p.value' based on permutation distribution of the
 maximum modification index per iteration: 

    lhs op   rhs     X2 df p.value tukey.p.value   flag            parameter
1  .p2. == .p34. 16.524  1   0.000          0.05         well_being=~hitop23
2  .p3. == .p35. 35.644  1   0.000          0.00 *  -->  well_being=~hitop54
3  .p4. == .p36.  0.053  1   0.817          1.00        well_being=~hitop106
4  .p5. == .p37.  0.346  1   0.556          1.00        well_being=~hitop149
5  .p6. == .p38.  0.900  1   0.343          1.00        well_being=~hitop200
6  .p7. == .p39.  2.734  1   0.098 

In [162]:
# let's do configural invariance for just one measure

In [163]:
cln_ss = 'anhedonic_depression'

In [174]:
# QUESTION
ro.globalenv[cln_ss+'_null'] = ro.StrVector(mdl)
ro.globalenv[cln_ss+'_mdl'] = mdl_strs_new[cln_ss]

In [175]:
# to load the data in R, let's save it as csv and then load it to R
# there may be a more elegant day to do this, but I don't know it...
mydata.to_csv('../data/CFA/mydata.csv')
#rread_csv = ro.r("read.csv")
ro.r("rdata <- read.csv('../data/CFA/mydata.csv', header=TRUE)")
rdata

,,X,hitop39,hitop77,hitop84,...,hitop281,whichdata
0,1,0,4,3,3,...,2,validation
1,2,1,2,1,1,...,3,validation
2,3,2,1,1,1,...,4,validation
3,4,3,1,1,1,...,1,validation
4,5,4,2,3,4,...,2,validation
5,6,5,3,2,3,...,3,validation
6,7,6,1,1,1,...,1,validation
7,8,7,1,1,2,...,1,validation
...,...,...,...,...,...,...,...,...
994,995,498,3,2,3,...,2,genpop


In [176]:
rdata

,,X,hitop39,hitop77,hitop84,...,hitop281,whichdata
0,1,0,4,3,3,...,2,validation
1,2,1,2,1,1,...,3,validation
2,3,2,1,1,1,...,4,validation
3,4,3,1,1,1,...,1,validation
4,5,4,2,3,4,...,2,validation
5,6,5,3,2,3,...,3,validation
6,7,6,1,1,1,...,1,validation
7,8,7,1,1,2,...,1,validation
...,...,...,...,...,...,...,...,...
994,995,498,3,2,3,...,2,genpop


In [190]:
#fit_null = ro.r(f'cfa({cln_ss}_null, data=rdata, group="whichdata", estimator="WLSMV")')

In [178]:
fit_config = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = "whichdata", estimator = "WLSMV")')

In [179]:
ro.r(f'{cln_ss}_mdl')

anhedonic_depression =~hitop39 + hitop77 + hitop84 + hitop92 + hitop93 + hitop123 + hitop157 + hitop182 + hitop230 + hitop246


In [180]:
ro.globalenv['fit_config'] = fit_config

In [181]:
# ignoring other types of invariance for now, let's just figure out config

In [182]:
lavaan = importr('lavaan')
semtools = importr('semTools')

#permuteMeasEq function - https://search.r-project.org/CRAN/refmans/semTools/html/permuteMeasEq.html
out_config = semtools.permuteMeasEq(nPermute=100, # 100 for now, 1000 for the "real" run
                                    con=fit_config, # In the case of testing configural invariance when modelType = "mgcfa", con is the configural model (implicitly, the unconstrained model is the saturated model, so use the defaults uncon = NULL and param = NULL).
                                    parallelType="multicore", ncpus=4)
# QUESTION why no null_fit?
# AFIs A character vector indicating which alternative fit indices (or chi-squared itself) are to be used to test the multiparameter omnibus null hypothesis that the constraints specified in con hold in the population. Any fit measures returned by fitMeasures may be specified (including constants like "df", which would be nonsensical). If both AFIs and moreAFIs are NULL, only "chisq" will be returned.
# In Dylan's code myAFIs = ro.StrVector(["chisq", "chisq.scaled", "cfi.robust", "tli.robust", "rmsea.robust"])
# I removed them and only running chisq (default)
# originally in Dylan's code moreAFIs=moreAFIs,I removed that for now


R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




In [173]:
ro.r('print(summary(fit_config, fit.measures=TRUE))')

lavaan 0.6.15 ended normally after 34 iterations

  Estimator                                       DWLS
  Optimization method                           NLMINB
  Number of model parameters                        60

  Number of observations per group:                   
    validation                                     496
    genpop                                         500

Model Test User Model:
                                              Standard      Scaled
  Test Statistic                                96.755     284.297
  Degrees of freedom                                70          70
  P-value (Chi-square)                           0.019       0.000
  Scaling correction factor                                  0.358
  Shift parameter                                           13.837
    simple second-order correction                                
  Test statistic for each group:
    validation                                  72.965     210.852
    genpop                

0,header,$lavaan.v...
1,optim,$estimato...
2,data,$ngroups ...
3,test,$standard...
4,fit,...
5,pe,...


In [187]:
fit_metric = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = "whichdata", estimator = "WLSMV", group.equal="loadings")')

In [192]:
ro.globalenv['fit_config'] = fit_config
ro.globalenv['fit_metric'] = fit_metric

In [193]:
ro.r('print(summary(fit_metric, fit.measures=TRUE))')

lavaan 0.6.15 ended normally after 28 iterations

  Estimator                                       DWLS
  Optimization method                           NLMINB
  Number of model parameters                        60
  Number of equality constraints                     9

  Number of observations per group:                   
    validation                                     496
    genpop                                         500

Model Test User Model:
                                              Standard      Scaled
  Test Statistic                               132.548     202.713
  Degrees of freedom                                79          79
  P-value (Chi-square)                           0.000       0.000
  Scaling correction factor                                  0.773
  Shift parameter                                           31.298
    simple second-order correction                                
  Test statistic for each group:
    validation                        

0,header,$lavaan.v...
1,optim,$estimato...
2,data,$ngroups ...
3,test,$standard...
4,fit,...
5,pe,...


In [195]:
out_metric = semtools.permuteMeasEq(nPermute=100, 
                                    uncon=fit_config, 
                                    con=fit_metric, 
                                    param="loadings",
                                    parallelType="multicore", ncpus=4)

R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




In [196]:
rprint = ro.r('print')
summary = ro.r('summary')
def print_summary(obj):
    return rprint(summary(obj))

rngkind = "L'Ecuyer-CMRG"

def lv_to_df(lv):
    names = lv.names
    names = [nn.replace('.', '_') for nn in names]

    df = pd.DataFrame(lv, index=names, columns=['val'])
    return df

def permmeaseq_to_df(permmeaseq):
    anova_res = lv_to_df(permmeaseq.slots['ANOVA'])
    anova_res.index = [ii+'_anova' for ii in anova_res.index]
    afi_dif = lv_to_df(permmeaseq.slots['AFI.obs'])
    afi_dif.index = [ii+'_diff' for ii in afi_dif.index]
    afi_ps = lv_to_df(permmeaseq.slots['AFI.pval'])
    afi_ps.index = [ii+'_p' for ii in afi_ps.index]
    return pd.concat([anova_res, afi_dif, afi_ps])

In [197]:
_ = print_summary(out_metric)

Omnibus p value based on parametric chi-squared difference test:

Chisq diff    Df diff Pr(>Chisq) 
    18.373      9.000      0.031 


Omnibus p values based on nonparametric permutation method: 

      AFI.Difference p.value
chisq         35.793    0.07


 Modification indices for equality constrained parameter estimates,
 with unadjusted 'p.value' based on chi-squared distribution and
 adjusted 'tukey.p.value' based on permutation distribution of the
 maximum modification index per iteration: 

    lhs op   rhs     X2 df p.value tukey.p.value flag
1  .p2. == .p34.  0.035  1   0.852          1.00     
2  .p3. == .p35.  1.101  1   0.294          0.99     
3  .p4. == .p36.  0.110  1   0.740          1.00     
4  .p5. == .p37. 15.274  1   0.000          0.06     
5  .p6. == .p38.  0.514  1   0.474          1.00     
6  .p7. == .p39.  0.420  1   0.517          1.00     
7  .p8. == .p40.  2.311  1   0.128          0.98     
8  .p9. == .p41.  1.628  1   0.202          0.99     
9 .p10. == 

In [198]:
fit_scalar = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = "whichdata", estimator = "WLSMV", group.equal=c("loadings", "intercepts"))')
out_scalar = semtools.permuteMeasEq(nPermute=100, 
                                    uncon=fit_metric, 
                                    con=fit_scalar, 
                                    param=ro.StrVector(["loadings","intercepts"]),
                                    parallelType="multicore", 
                                    ncpus=4)
_ = print_summary(out_scalar)

R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




Omnibus p value based on parametric chi-squared difference test:

Chisq diff    Df diff Pr(>Chisq) 
    71.849      9.000      0.000 


Omnibus p values based on nonparametric permutation method: 

      AFI.Difference p.value
chisq         48.689       0


 Modification indices for equality constrained parameter estimates,
 with unadjusted 'p.value' based on chi-squared distribution and
 adjusted 'tukey.p.value' based on permutation distribution of the
 maximum modification index per iteration: 

     lhs op   rhs     X2 df p.value tukey.p.value   flag
1   .p2. == .p34.  0.041  1   0.839          1.00       
2   .p3. == .p35.  4.310  1   0.038          0.68       
3   .p4. == .p36.  0.219  1   0.640          1.00       
4   .p5. == .p37. 14.364  1   0.000          0.09       
5   .p6. == .p38.  0.400  1   0.527          1.00       
6   .p7. == .p39.  0.191  1   0.662          1.00       
7   .p8. == .p40.  1.915  1   0.166          0.99       
8   .p9. == .p41.  2.141  1   0.143      

In [199]:
fit_strict = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = "whichdata", estimator = "WLSMV", group.equal=c("loadings", "intercepts", "residuals"))')
out_strict = semtools.permuteMeasEq(nPermute=100, 
                                    uncon=fit_scalar, 
                                    con=fit_strict, 
                                    param=ro.StrVector(["loadings","intercepts", "residuals"]),
                                    parallelType="multicore", 
                                    ncpus=4)
_ = print_summary(out_strict)

R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




Omnibus p value based on parametric chi-squared difference test:

Chisq diff    Df diff Pr(>Chisq) 
    78.961     10.000      0.000 


Omnibus p values based on nonparametric permutation method: 

      AFI.Difference p.value
chisq         58.015       0


 Modification indices for equality constrained parameter estimates,
 with unadjusted 'p.value' based on chi-squared distribution and
 adjusted 'tukey.p.value' based on permutation distribution of the
 maximum modification index per iteration: 

     lhs op   rhs     X2 df p.value tukey.p.value   flag
1   .p2. == .p34.  0.989  1   0.320          1.00       
2   .p3. == .p35.  6.794  1   0.009          0.50       
3   .p4. == .p36.  0.029  1   0.866          1.00       
4   .p5. == .p37. 18.186  1   0.000          0.07       
5   .p6. == .p38.  0.002  1   0.963          1.00       
6   .p7. == .p39.  1.942  1   0.163          1.00       
7   .p8. == .p40.  3.348  1   0.067          0.92       
8   .p9. == .p41.  4.074  1   0.044      